In [6]:
import pandas as pd
import os
import cv2
import shutil
import pydicom
from PIL import Image
import numpy as np
import glob
import math

# 選出Mass的圖片

In [5]:
file_path = 'D:/vindr-mammo/finding_annotations.csv'
source_folder = "D:/vindr-mammo/process_new2/image_png"
target_folder = "D:/vindr-mammo/process_new2/Mass/original_image"

df = pd.read_csv(file_path)

data = df.values.tolist()

for i in range (len(data)):
    if data[i][9] == "['Mass']":
        source_file_path = os.path.join(source_folder, data[i][2]+'.png')
        target_file_path = os.path.join(target_folder, data[i][2]+'.png')
        shutil.copy(source_file_path, target_file_path)
        
    else:
        continue
        
print("finished")

finished


# 選出Norm的圖片

In [7]:
file_path = 'D:/vindr-mammo/finding_annotations.csv'
source_folder = "D:/vindr-mammo/process_new2/image_png"
target_folder = "D:/vindr-mammo/process_new2/Norm/original_image"

df = pd.read_csv(file_path)

data = df.values.tolist()

for i in range (len(data)):
    if data[i][9] == "['No Finding']":
        source_file_path = os.path.join(source_folder, data[i][2]+'.png')
        target_file_path = os.path.join(target_folder, data[i][2]+'.png')
        shutil.copy(source_file_path, target_file_path)
        
    else:
        continue
        
print("finished")

finished


# 產生label

In [7]:
output_folder = "G:/BC_dataset/VinDr_Mammo/process_new2/Mass/process_image/label"
annotation_file_path = 'G:/BC_dataset/VinDr_Mammo/finding_annotations_mass.csv'
input_folder = "G:/BC_dataset/VinDr_Mammo/process_new2/Mass/original"
png_files = glob.glob(os.path.join(input_folder, "*.png"))
df = pd.read_csv(annotation_file_path)
data = df.values.tolist()

for png_file in png_files:
    image = cv2.imread(png_file)
    file_name, file_extension = os.path.splitext(os.path.basename(png_file))
    for i in range(len(data)):
        if data[i][2] == file_name:
            xmin = int(data[i][11])
            ymin = int(data[i][12])
            xmax = int(math.ceil(data[i][13]))
            ymax = int(math.ceil(data[i][14]))

            height,width,_ = image.shape

            x_center = (xmin + xmax) / (2 * width)
            y_center = (ymin + ymax) / (2 * height)
            box_width = (xmax - xmin) / width
            box_height = (ymax - ymin) / height

            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            
            output_image_path = os.path.join(output_folder, f"{file_name}{file_extension}")
            cv2.imwrite(output_image_path, image)
            
print("finished")
    

finished


# 去除部分背景，並產生新的label(Mass)

In [14]:
output_folder = "D:/vindr-mammo/process_new2/Mass/process_image/cut"
output_label_folder = "D:/vindr-mammo/process_new2/Mass/process_image/cut_label"
annotation_file_path = 'D:/vindr-mammo/process_new/Mass/finding_annotations_mass.csv'
input_folder = "D:/vindr-mammo/process_new2/Mass/original_image"
png_files = glob.glob(os.path.join(input_folder, "*.png"))
df = pd.read_csv(annotation_file_path)
data = df.values.tolist()


for png_file in png_files:
    image = cv2.imread(png_file)

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gray_image[gray_image>=160]=0

    _, thresholded = cv2.threshold(gray_image, 100, 255, cv2.THRESH_BINARY)

    coords = cv2.findNonZero(thresholded)

    x, y, width, height = cv2.boundingRect(coords)

    cropped_image = image[y:y+height, x:x+width]

    gray_cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

    file_name, file_extension = os.path.splitext(os.path.basename(png_file))

    processed_image_path = os.path.join(output_folder, f"{file_name}{file_extension}")

    cv2.imwrite(processed_image_path, cropped_image)

    i=0

    num = 0
    for i in range(len(data)):
        if data[i][2] == file_name:
            xmin = int(data[i][11] - float(x))
            ymin = int(data[i][12] - float(y))
            xmax = int(math.ceil(data[i][13] - float(x)))
            ymax = int(math.ceil(data[i][14] - float(y)))

            height,width,_ = cropped_image.shape

            x_center = (xmin + xmax) / (2 * width)
            y_center = (ymin + ymax) / (2 * height)
            box_width = (xmax - xmin) / width
            box_height = (ymax - ymin) / height

            cv2.rectangle(cropped_image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            output_image_path = os.path.join(output_label_folder, f"{file_name}{file_extension}")
            cv2.imwrite(output_image_path, cropped_image)

            label=""
            if(data[i][10] == 'BI-RADS 3'):
                label_type = 0
                label += f"{label_type} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n"

            else:
                label_type = 0
                label += f"{label_type} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n"


            if(num != 0):
                output_label_path = os.path.join(output_folder, f"{file_name}.txt")
                with open(output_label_path, 'a') as label_file:
                    label_file.write(label)

            else:
                output_label_path = os.path.join(output_folder, f"{file_name}.txt")
                with open(output_label_path, 'w') as label_file:
                    label_file.write(label)

            num = 1
             
print("finished")

finished


## 去除部分背景，並產生新的label(NORM)

In [5]:
NORM_folder = "D:/vindr-mammo/process_new2/Norm/process_image/cut"
annotation_file_path = 'D:/vindr-mammo/finding_annotations.csv'
original_image_folder = "D:/vindr-mammo/process_new2/Norm/original_image"
png_files = glob.glob(os.path.join(original_image_folder, "*.png"))
df = pd.read_csv(annotation_file_path)

data = df.values.tolist()

for png_file in png_files:
    image = cv2.imread(png_file)

    file_name, file_extension = os.path.splitext(os.path.basename(png_file))
    
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gray_image[gray_image>=160]=0

    _, thresholded = cv2.threshold(gray_image, 141, 255, cv2.THRESH_BINARY)

    coords = cv2.findNonZero(thresholded)

    x, y, width, height = cv2.boundingRect(coords)

    cropped_image = image[y:y+height, x:x+width]

    gray_cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

    file_name, file_extension = os.path.splitext(os.path.basename(png_file))

    processed_image_path = os.path.join(NORM_folder, f"{file_name}{file_extension}")

    cv2.imwrite(processed_image_path, cropped_image)

    label=""
    label += f"\n"

    output_label_path = os.path.join(NORM_folder, f"{file_name}.txt")
    with open(output_label_path, 'w') as label_file:
        label_file.write(label)
             
print("finished")


finished


# 選出calcification的圖片

In [11]:
file_path = 'D:/vindr-mammo/finding_annotations.csv'
source_folder = "D:/vindr-mammo/process_new2/image_png"
target_folder = "D:/vindr-mammo/process_new2/Calcification/original_image"

df = pd.read_csv(file_path)

data = df.values.tolist()

for i in range (len(data)):
    if data[i][9] == "['Suspicious Calcification']":
        source_file_path = os.path.join(source_folder, data[i][2]+'.png')
        target_file_path = os.path.join(target_folder, data[i][2]+'.png')
        shutil.copy(source_file_path, target_file_path)
        
    else:
        continue
        
print("finished")

finished


# 去除部分背景，並產生新的label(Calcification)

In [14]:
output_folder = "D:/vindr-mammo/process_new2/Calcification/process_image/cut"
output_label_folder = "D:/vindr-mammo/process_new2/Calcification/process_image/cut_label"
annotation_file_path = 'D:/vindr-mammo/process_new/Calcification/finding_annotations_calcification.csv'
input_folder = "D:/vindr-mammo/process_new2/Calcification/original_image"
png_files = glob.glob(os.path.join(input_folder, "*.png"))
df = pd.read_csv(annotation_file_path)
data = df.values.tolist()

for png_file in png_files:
    image = cv2.imread(png_file)

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gray_image[gray_image>=160]=0

    _, thresholded = cv2.threshold(gray_image, 141, 255, cv2.THRESH_BINARY)

    coords = cv2.findNonZero(thresholded)

    x, y, width, height = cv2.boundingRect(coords)

    cropped_image = image[y:y+height, x:x+width]

    gray_cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

    file_name, file_extension = os.path.splitext(os.path.basename(png_file))

    processed_image_path = os.path.join(output_folder, f"{file_name}{file_extension}")

    cv2.imwrite(processed_image_path, cropped_image)

    i=0

    num = 0
    for i in range(len(data)):
        if data[i][2] == file_name:
            xmin = int(data[i][11] - float(x))
            ymin = int(data[i][12] - float(y))
            xmax = int(math.ceil(data[i][13] - float(x)))
            ymax = int(math.ceil(data[i][14] - float(y)))

            height,width = gray_cropped_image.shape

            x_center = (xmin + xmax) / (2 * width)
            y_center = (ymin + ymax) / (2 * height)
            box_width = (xmax - xmin) / width
            box_height = (ymax - ymin) / height

            cv2.rectangle(cropped_image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            output_image_path = os.path.join(output_label_folder, f"{file_name}{file_extension}")
            cv2.imwrite(output_image_path, cropped_image)

            label=""
            if(data[i][10] == 'BI-RADS 3'):
                label_type = 0
                label += f"{label_type} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n"

            else:
                label_type = 0
                label += f"{label_type} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n"


            if(num != 0):
                output_label_path = os.path.join(output_folder, f"{file_name}.txt")
                with open(output_label_path, 'a') as label_file:
                    label_file.write(label)

            else:
                output_label_path = os.path.join(output_folder, f"{file_name}.txt")
                with open(output_label_path, 'w') as label_file:
                    label_file.write(label)

            num = 1
             
print("finished")

finished


# 分配YOLO Train及Valid資料

In [4]:
import glob
import os
from os import listdir, getcwd
import random
import numpy as np

input_path = 'D:/darknet-master/build/darknet/x64/YOLOs'
cfg_path = 'D:/darknet-master/build/darknet/x64/data/'
train_ratio = 0.8

if os.path.isdir(input_path):  
    types = os.path.join(input_path,'*.jpg'), os.path.join(input_path,'*.jpeg'), os.path.join(input_path,'*.png')
    files_grabbed = []
    for files in types:
        files_grabbed.extend(sorted(glob.iglob(files)))  
elif os.path.isfile(input_path):  
    files_grabbed = [input_path]
else:  
    raise ValueError("File PATH is NOT Valid")        

train_file = open('%s/%s.txt' % (cfg_path, 'train'), 'w')
test_file = open('%s/%s.txt' % (cfg_path, 'valid'), 'w')

trainNum = int(len(files_grabbed) * train_ratio)


random.shuffle(files_grabbed)

train_file.write("\n".join(files_grabbed[:trainNum]))
test_file.write("\n".join(files_grabbed[trainNum:]))

3475

# PHASE_1 strategy2

In [27]:
txt_folder = "C:/Users/home/Desktop/Two_FULL_datasets/PHASE_1/strategy_2/S2_datasets"
txt_files = glob.glob(os.path.join(txt_folder, "*.txt"))

for txt_file in txt_files:
    file_name, file_extension = os.path.splitext(os.path.basename(txt_file))
    label = ""
    with open(txt_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
        
        if parts[0]=="1":
            label_type = '0'
            label += f"{label_type} {parts[1]} {parts[2]} {parts[3]} {parts[4]}\n"
            
            output_label_path = os.path.join(txt_folder, f"{file_name}.txt")
            with open(output_label_path, 'w') as label_file:
                label_file.write(label)

        else:
            continue

print("finished")  

finished


# 影像切割成slice

## 1024*1024

In [9]:
def calculate_zero_pixel_ratio(image):
    total_pixels = image.size
    zero_pixels = np.count_nonzero(image == 0)
    zero_ratio = zero_pixels / total_pixels
    return zero_ratio

def process_image(image_path,dataset_folder):
    image = cv2.imread(image_path)
    file_name, file_extension = os.path.splitext(os.path.basename(image_path))
    label_path = os.path.join(dataset_folder,f"{file_name}.txt")
    original_height, original_width, _ = image.shape
    slice_size = 1024
    overlap = int(slice_size * 0.5)
    
    with open(label_path, 'r') as label_file:
        lines = label_file.readlines()
        
    num = 0
    
    for slice_y in range(0, original_height -  slice_size + 1, overlap):
        for slice_x in range(0, original_width - slice_size + 1, overlap):
            slice_image = image[slice_y:slice_y+slice_size, slice_x:slice_x+slice_size]
            zero_ratio = calculate_zero_pixel_ratio(slice_image)
            if(zero_ratio>0.75):
                continue
            new_label_lines = []
            for line in lines:
                label_info = line.strip().split()
                class_id = label_info[0]
                x_center = float(label_info[1])
                y_center = float(label_info[2])
                width = float(label_info[3])
                height = float(label_info[4])
                
                label_x_min = math.floor((x_center * original_width) - (width * original_width / 2))
                label_y_min = math.floor((y_center * original_height) - (height * original_height / 2))
                label_x_max = math.ceil(label_x_min + width * original_width)
                label_y_max = math.ceil(label_y_min + height * original_height)
                
                slice_x_min = slice_x
                slice_y_min = slice_y
                slice_x_max = slice_x + slice_size
                slice_y_max = slice_y + slice_size
                
                overlap_x_min = max(label_x_min, slice_x_min)
                overlap_y_min = max(label_y_min, slice_y_min)
                overlap_x_max = min(label_x_max, slice_x_max)
                overlap_y_max = min(label_y_max, slice_y_max)
                if overlap_x_min < overlap_x_max and overlap_y_min < overlap_y_max:
                    new_width = (overlap_x_max - overlap_x_min) / slice_size
                    new_height = (overlap_y_max - overlap_y_min) / slice_size
                    new_x_center = (((overlap_x_min + overlap_x_max) / 2) - slice_x_min) / slice_size
                    new_y_center = (((overlap_y_min + overlap_y_max) / 2) - slice_y_min) / slice_size
                    
                    if((new_width*slice_size)*(new_height*slice_size))/((width*original_width)*(height*original_height))<0.7:
                        continue
                    
                    slice_folder = "D:/vindr-mammo/process_new2/Mass/process_image/1024_1024/311/AN"
                    slice_filename = os.path.join(slice_folder,f"{file_name}_slice{num}.png")
                    cv2.imwrite(slice_filename, slice_image)
                    
                    new_x_min = math.floor((new_x_center*slice_size) - (new_width*slice_size/2))
                    new_y_min = math.floor((new_y_center*slice_size) - (new_height*slice_size/2))
                    new_x_max = math.ceil((new_x_center*slice_size) + (new_width*slice_size/2))
                    new_y_max = math.ceil((new_y_center*slice_size) + (new_height*slice_size/2))
                    
                    label_picture = cv2.imread(slice_filename)
                    cv2.rectangle(label_picture, (new_x_min, new_y_min), (new_x_max, new_y_max), (0, 255, 0), 2)
                    
                    output_label_png_folder = "D:/vindr-mammo/process_new2/Mass/process_image/1024_1024/311/label"
                    output_image_path = os.path.join(output_label_png_folder, f"{file_name}_slice{num}_label.png")
                    cv2.imwrite(output_image_path, label_picture)
                    
                    new_label_lines.append(f"{class_id} {new_x_center:.6f} {new_y_center:.6f} {new_width:.6f} {new_height:.6f}")
                
                    txt_folder = "D:/vindr-mammo/process_new2/Mass/process_image/1024_1024/311/AN"
                    output_label_path = os.path.join(txt_folder, f"{file_name}_slice{num}.txt")
                    with open(output_label_path, 'w') as new_label_file:
                        new_label_file.write('\n'.join(new_label_lines))
                    
                    num = num + 1
                else :
                    slice_folder = "D:/vindr-mammo/process_new2/Mass/process_image/1024_1024/311/N"
                    slice_filename = os.path.join(slice_folder,f"{file_name}_slice{num}.png")
                    cv2.imwrite(slice_filename, slice_image)
                    
                    txt_folder = "D:/vindr-mammo/process_new2/Mass/process_image/1024_1024/311/N"
                    output_label_path = os.path.join(txt_folder, f"{file_name}_slice{num}.txt")
                    with open(output_label_path, 'w') as new_label_file:
                        new_label_file.write("")
                        
                    num = num + 1

if __name__ == "__main__":
    dataset_folder = "D:/vindr-mammo/process_new2/Mass/process_image/311"
    png_files = glob.glob(os.path.join(dataset_folder, "*.png"))
    for png_file in png_files:
        image = cv2.imread(png_file)
        original_height, original_width, _ = image.shape
        slice_size = 1024
        if(original_height < slice_size or original_width < slice_size):
            continue
        else:
            process_image(png_file, dataset_folder)
    
    print("finished")

finished


In [61]:
txt_folder = "G:/BC_dataset/Two_FULL_datasets/PHASE_2/1024x1024/mass"
txt_files = glob.glob(os.path.join(txt_folder, "*.txt"))

for txt_file in txt_files:
    file_name, file_extension = os.path.splitext(os.path.basename(txt_file))
    label = ""
    with open(txt_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
        
        if parts[0]=="1":
            label_type = '0'
            label += f"{label_type} {parts[1]} {parts[2]} {parts[3]} {parts[4]}\n"
            
            output_label_path = os.path.join(txt_folder, f"{file_name}.txt")
            with open(output_label_path, 'w') as label_file:
                label_file.write(label)

        else:
            continue

print("finished")  

finished


In [44]:
def calculate_zero_pixel_ratio(image):
    total_pixels = image.size
    zero_pixels = np.count_nonzero(image == 0)
    zero_ratio = zero_pixels / total_pixels
    return zero_ratio

def process_image(image_path, label_path):
    image = cv2.imread(image_path)
    file_name, file_extension = os.path.splitext(os.path.basename(image_path))
    original_height, original_width, _ = image.shape
    slice_size = 1024
    overlap = int(slice_size * 0.5)
    
    with open(label_path, 'r') as label_file:
        lines = label_file.readlines()
        
    num = 0
    
    for slice_y in range(0, original_height -  slice_size + 1, overlap):
        for slice_x in range(0, original_width - slice_size + 1, overlap):
            slice_image = image[slice_y:slice_y+slice_size, slice_x:slice_x+slice_size]
            zero_ratio = calculate_zero_pixel_ratio(slice_image)
            new_label_lines = []
            for line in lines:
                label_info = line.strip().split()
                class_id = label_info[0]
                x_center = float(label_info[1])
                y_center = float(label_info[2])
                width = float(label_info[3])
                height = float(label_info[4])
                
                label_x_min = math.floor((x_center * original_width) - (width * original_width / 2))
                label_y_min = math.floor((y_center * original_height) - (height * original_height / 2))
                label_x_max = math.ceil(label_x_min + width * original_width)
                label_y_max = math.ceil(label_y_min + height * original_height)
                
                slice_x_min = slice_x
                slice_y_min = slice_y
                slice_x_max = slice_x + slice_size
                slice_y_max = slice_y + slice_size
                
                overlap_x_min = max(label_x_min, slice_x_min)
                overlap_y_min = max(label_y_min, slice_y_min)
                overlap_x_max = min(label_x_max, slice_x_max)
                overlap_y_max = min(label_y_max, slice_y_max)
                if overlap_x_min < overlap_x_max and overlap_y_min < overlap_y_max:
                    new_width = (overlap_x_max - overlap_x_min) / slice_size
                    new_height = (overlap_y_max - overlap_y_min) / slice_size
                    new_x_center = (((overlap_x_min + overlap_x_max) / 2) - slice_x_min) / slice_size
                    new_y_center = (((overlap_y_min + overlap_y_max) / 2) - slice_y_min) / slice_size
                    
                    slice_folder = "C:/Users/home/Desktop/Two_FULL_datasets/PHASE_2/slice"
                    slice_filename = os.path.join(slice_folder,f"{file_name}_slice{num}.png")
                    cv2.imwrite(slice_filename, slice_image)
                    
                    new_x_min = math.floor((new_x_center*slice_size) - (new_width*slice_size/2))
                    new_y_min = math.floor((new_y_center*slice_size) - (new_height*slice_size/2))
                    new_x_max = math.ceil((new_x_center*slice_size) + (new_width*slice_size/2))
                    new_y_max = math.ceil((new_y_center*slice_size) + (new_height*slice_size/2))
                    
                    label_picture = cv2.imread(slice_filename)
                    cv2.rectangle(label_picture, (new_x_min, new_y_min), (new_x_max, new_y_max), (0, 255, 0), 2)
                    
                    output_label_png_folder = "C:/Users/home/Desktop/Two_FULL_datasets/PHASE_2/slice_label"
                    output_image_path = os.path.join(output_label_png_folder, f"{file_name}_slice{num}_label.png")
                    cv2.imwrite(output_image_path, label_picture)
                    
                    new_label_lines.append(f"{class_id} {new_x_center:.6f} {new_y_center:.6f} {new_width:.6f} {new_height:.6f}")
                
                    txt_folder = "C:/Users/home/Desktop/Two_FULL_datasets/PHASE_2/slice_label"
                    output_label_path = os.path.join(txt_folder, f"{file_name}_slice{num}_label.txt")
                    with open(output_label_path, 'w') as new_label_file:
                        new_label_file.write('\n'.join(new_label_lines))
                    
                    num = num + 1

if __name__ == "__main__":
    image_path = "C:/Users/home/Desktop/Two_FULL_datasets/PHASE_2/00be38a5c0566291168fe381ba0028e6.png"
    label_path = "C:/Users/home/Desktop/Two_FULL_datasets/PHASE_2/00be38a5c0566291168fe381ba0028e6.txt"
    
    process_image(image_path, label_path)
    print("finished")

finished


# PHASE_3

## 切割lesion區域

In [3]:
annotation_file = "C:/Users/home/Desktop/Two_FULL_datasets/finding_annotations_mass.csv"
image_folder = "C:/Users/home/Desktop/Two_FULL_datasets/M"
benign = "G:/BC_dataset/Two_FULL_datasets/PHASE_3/benign"
malignant = "G:/BC_dataset/Two_FULL_datasets/PHASE_3/malignant"

df = pd.read_csv(annotation_file)

datas = df.values.tolist()

for data in datas:
    xmin = math.floor(data[11])
    ymin = math.floor(data[12])
    xmax = math.ceil(data[13])
    ymax = math.ceil(data[14])
    finding_birads = data[10]
    
    png_file = os.path.join(image_folder , f"{data[2]}.png")
    image = cv2.imread(png_file)
    process_image = image[ymin:ymax,xmin:xmax]

    if(finding_birads=="BI-RADS 4" or finding_birads=="BI-RADS 5"):
        output = os.path.join(malignant , f"{data[2]}.png")
        cv2.imwrite(output , process_image)
    
    else:
        output = os.path.join(benign , f"{data[2]}.png")
        cv2.imwrite(output , process_image)
        
print("finished")

finished


# 完整執行偵測圖片 

## FMD

In [9]:
import subprocess
import time

path_to_open = "D:/darknet-master/build/darknet/x64"

command_to_execute = "darknet.exe detector test data/obj_FMD_test.data cfg/FMD-test.cfg weights/P1/F4_no_negative.weights -dont_show -ext_output < data/FMD_test.txt > results/FMD_result.txt"

try:
    subprocess.Popen(f'cmd /c "cd /d {path_to_open} && {command_to_execute}"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    print("已執行命令。")
except Exception as e:
    print("發生錯誤:", str(e))

time.sleep(10)
print("FMD_result.txt complete!")

已執行命令。
FMD_result.txt complete!


In [7]:
file_path = "D:/darknet-master/build/darknet/x64/results/FMD_result.txt"
FMD_data = []
FMD_result = []
with open(file_path, 'r') as file:
        lines = file.readlines()

for line in lines:
    if "C:/Users/HAUYU/Desktop/data/full" in line or "abnormal" in line:
         FMD_data.append(line)


num = 0
for data in FMD_data:
    if(num==0):
        colon_index = data.find(': Predicted')
        data = data[:colon_index]
        basename = os.path.basename(data).split('.')[0]
        picture_data=[basename]
        num = num + 1
    else:
        if(data.startswith('C:/Users/HAUYU/Desktop/data/full')):
            FMD_result.append(picture_data)
            picture_data = []
            colon_index = data.find(': Predicted')
            data = data[:colon_index]
            basename = os.path.basename(data).split('.')[0]
            picture_data=[basename]
            num = num + 1
            if(num == len(FMD_data)):
                FMD_result.append(picture_data)
        else:
            values = []
            values.extend(data.strip().split())
            values[1] = float(values[1].replace('%', '')) / 100 
            picture_data.append(str(values[1]))
            picture_data.append(values[3])
            picture_data.append(values[5])
            picture_data.append(str(int(values[3])+int(values[7])))
            values[9] = int(values[9].replace(')', ''))
            picture_data.append(str(int(values[5])+int(values[9])))
            num = num + 1
            if(num == len(FMD_data)):
                FMD_result.append(picture_data)
print(len(FMD_result))
print(FMD_result)

106
[['20586908_6c613a14b80a8591_MG_R_CC_ANON'], ['20586934_6c613a14b80a8591_MG_L_CC_ANON', '0.84', '99', '2004', '330', '2234', '0.66', '616', '1130', '842', '1333'], ['20586960_6c613a14b80a8591_MG_R_ML_ANON', '0.26', '1795', '1275', '1898', '1408'], ['20586986_6c613a14b80a8591_MG_L_ML_ANON'], ['20587612_f4b2d377f43ba0bd_MG_R_CC_ANON', '0.68', '2769', '529', '3032', '767'], ['20587664_f4b2d377f43ba0bd_MG_R_ML_ANON', '0.55', '2378', '840', '2763', '1173'], ['20587758_81cd83d2f4d78528_MG_L_CC_ANON', '0.99', '188', '1068', '379', '1281'], ['20587810_81cd83d2f4d78528_MG_L_ML_ANON'], ['20587902_8dbbd4e51f549ff0_MG_R_CC_ANON', '0.98', '1528', '1084', '2411', '1752'], ['20587928_8dbbd4e51f549ff0_MG_R_ML_ANON', '0.94', '1366', '1279', '2108', '1987'], ['20587994_024ee3569b2605dc_MG_R_CC_ANON'], ['20588046_024ee3569b2605dc_MG_R_ML_ANON', '0.98', '1980', '2184', '2250', '2367'], ['20588190_8d0b9620c53c0268_MG_L_CC_ANON', '0.99', '83', '850', '289', '1105'], ['20588216_8d0b9620c53c0268_MG_L_ML_A

## CMD

### 剪裁測試圖片

In [8]:
def calculate_zero_pixel_ratio(image):
    total_pixels = image.size
    zero_pixels = np.count_nonzero(image == 0)
    zero_ratio = zero_pixels / total_pixels
    return zero_ratio
                                      
if __name__ == "__main__":
    dataset_folder = "C:/Users/HAUYU/Desktop/data/full"
    cropped_path = "C:/Users/HAUYU/Desktop/data/cropped"
    CMD_result = []
    png_files = glob.glob(os.path.join(dataset_folder, "*.png"))
    for png_file in png_files:
        image = cv2.imread(png_file)
        original_height, original_width, _ = image.shape
        slice_size = 1024
        if(original_height < slice_size or original_width < slice_size):
            continue
        else:
            file_name = os.path.basename(png_file).split('.')[0]
            original_height, original_width, _ = image.shape
            slice_size = 1024
            overlap = int(slice_size * 0.5)
            num = 0
            for slice_y in range(0, original_height -  slice_size + 1, overlap):
                for slice_x in range(0, original_width - slice_size + 1, overlap):
                    slice_image = image[slice_y:slice_y+slice_size, slice_x:slice_x+slice_size]
                    zero_ratio = calculate_zero_pixel_ratio(slice_image)
                    if(zero_ratio>0.75):
                        continue          
                    slice_x_min = slice_x
                    slice_y_min = slice_y
                    slice_x_max = slice_x + slice_size
                    slice_y_max = slice_y + slice_size

                    slice_folder = os.path.join(cropped_path,file_name)
                    if not os.path.exists(slice_folder):
                        os.makedirs(slice_folder)
                    slice_filename = os.path.join(slice_folder,f"{file_name}_slice{num}.png")
                    cv2.imwrite(slice_filename, slice_image)
                    slice_basename = os.path.basename(slice_filename).split('.')[0]
                    CMD_data = [slice_basename,str(slice_x_min),str(slice_y_min),str(slice_x_max),str(slice_y_max)]
                    CMD_result.append(CMD_data)
                    num = num + 1
    
    cropped_folder = "C:/Users/HAUYU/Desktop/data/cropped"
    CMD_test = "D:/darknet-master/build/darknet/x64/data/CMD_test.txt"
    subfolders = [f.path for f in os.scandir(cropped_folder) if f.is_dir()]
    with open(CMD_test, 'w') as file:
        for subfolder in subfolders:
            subfolder_name = os.path.basename(subfolder)
            png_files = [f.path for f in os.scandir(subfolder) if f.is_file() and f.name.endswith('.png')]
            for png_file in png_files:
                file.write(png_file + '\n')       

In [18]:
path_to_open = "D:/darknet-master/build/darknet/x64"

command_to_execute = "darknet.exe detector test data/obj_CMD_test.data cfg/CMD-test.cfg weights/P2/yolo-obj_best.weights -dont_show -ext_output < data/CMD_test.txt > results/CMD_result.txt"

try:
    subprocess.Popen(f'cmd /c "cd /d {path_to_open} && {command_to_execute}"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    print("已執行命令。")
except Exception as e:
    print("發生錯誤:", str(e))

time.sleep(10)
print("CMD_result.txt complete!")

已執行命令。
CMD_result.txt complete!


In [10]:
file_path = "D:/darknet-master/build/darknet/x64/results/CMD_result.txt"
with open(file_path, 'r') as file:
        lines = file.readlines()

CMD_data = []
for line in lines:
    if "C:/Users/HAUYU/Desktop/data/cropped" in line or "abnormal" in line:
         CMD_data.append(line)

num = 0
for data in CMD_data:
    if(num == 0):
        colon_index = data.find(': Predicted')
        data = data[:colon_index]
        basename = os.path.basename(data).split('.')[0]
        picture_data = []
        num = num + 1
    else:
        if(data.startswith('C:/Users/HAUYU/Desktop/data/cropped')):
            for i in range(len(CMD_result)):
                if(basename == CMD_result[i][0]):
                    for j in range(len(picture_data)):
                        CMD_result[i].append(picture_data[j])
            colon_index = data.find(': Predicted')
            data = data[:colon_index]
            basename = os.path.basename(data).split('.')[0]
            picture_data = []
        elif(data.startswith('abnormal')):
            values = []
            values.extend(data.strip().split())
            values[1] = float(values[1].replace('%', '')) / 100 
            picture_data.append(str(values[1]))
            picture_data.append(values[3])
            picture_data.append(values[5])
            picture_data.append(values[7])
            values[9] = int(values[9].replace(')', ''))
            picture_data.append(str(values[9]))
print(CMD_result)

[['20586908_6c613a14b80a8591_MG_R_CC_ANON_slice0', '1536', '0', '2560', '1024', '0.26', '41', '-224', '930', '920', '0.26', '513', '713', '400', '239'], ['20586908_6c613a14b80a8591_MG_R_CC_ANON_slice1', '2048', '0', '3072', '1024', '0.56', '45', '706', '365', '252'], ['20586908_6c613a14b80a8591_MG_R_CC_ANON_slice2', '1024', '512', '2048', '1536'], ['20586908_6c613a14b80a8591_MG_R_CC_ANON_slice3', '1536', '512', '2560', '1536'], ['20586908_6c613a14b80a8591_MG_R_CC_ANON_slice4', '2048', '512', '3072', '1536', '0.38', '42', '202', '374', '226'], ['20586908_6c613a14b80a8591_MG_R_CC_ANON_slice5', '1024', '1024', '2048', '2048'], ['20586908_6c613a14b80a8591_MG_R_CC_ANON_slice6', '1536', '1024', '2560', '2048'], ['20586908_6c613a14b80a8591_MG_R_CC_ANON_slice7', '2048', '1024', '3072', '2048', '0.62', '325', '1', '129', '89', '0.68', '393', '588', '152', '151'], ['20586908_6c613a14b80a8591_MG_R_CC_ANON_slice8', '1024', '1536', '2048', '2560'], ['20586908_6c613a14b80a8591_MG_R_CC_ANON_slice9', 

In [11]:
cropped_folder = "C:/Users/HAUYU/Desktop/data/cropped"
subfolders = [f.path for f in os.scandir(cropped_folder) if f.is_dir()]
result_list = []
for subfolder in subfolders:
    foldername = os.path.basename(subfolder).split('.')[0]
    overall_data = []
    for sublist in CMD_result:
        for element in sublist:
            if foldername in element:
                if(len(sublist)>5):
                    overall_data.append(sublist)
    result_list.append(overall_data)

overall_data = []
num=0
for subfolder in subfolders:
    foldername = os.path.basename(subfolder).split('.')[0]
    picture_data = []
    element=0
    picture_data.append(f"{foldername}")
    for i in range(len(result_list[num])):
        index = int((len(result_list[num][element])-1)/5)
        for j in range(index):
            if(float(result_list[num][element][(j+1)*5])>0.5):
                xmin = int(result_list[num][element][1]) + int(result_list[num][element][(j+1)*5+1])
                ymin = int(result_list[num][element][2]) + int(result_list[num][element][(j+1)*5+2])
                xmax = xmin + int(result_list[num][element][(j+1)*5+3])
                ymax = ymin + int(result_list[num][element][(j+1)*5+4])
                picture_data.append(str(result_list[num][element][(j+1)*5]))
                picture_data.append(str(xmin))
                picture_data.append(str(ymin))
                picture_data.append(str(xmax))
                picture_data.append(str(ymax))
            else:
                continue
        element = element + 1
    overall_data.append(picture_data)
    num = num + 1

CMD_result = []
for i in range(len(overall_data)):
    data_number = int((len(overall_data[i])-1)/5)
    if(data_number<=1):
        CMD_result.append(overall_data[i])
        continue
    else:
        data_list = []
        num = 0
        for j in range(data_number):
            data_list.append([overall_data[i][1+num],overall_data[i][2+num],overall_data[i][3+num],overall_data[i][4+num],overall_data[i][5+num]])
            num = num + 5
        if data_number==2:
            confidence_i, xmin_i, ymin_i, xmax_i, ymax_i = data_list[0]
            confidence_j, xmin_j, ymin_j, xmax_j, ymax_j = data_list[1]
            if (float(xmin_i) <= float(xmax_j) and float(xmax_i) >= float(xmin_j) and float(ymin_i) <= float(ymax_j) and float(ymax_i) >= float(ymin_j)):
                merged_xmin = min(int(xmin_i), int(xmin_j))
                merged_ymin = min(int(ymin_i), int(ymin_j))
                merged_xmax = max(int(xmax_i), int(xmax_j))
                merged_ymax = max(int(ymax_i), int(ymax_j))
                merged_confidence = max(float(confidence_i), float(confidence_j))
                
                name = overall_data[i][0]
                overall_data[i].clear()
                overall_data[i].append(name)
                overall_data[i].append(str(merged_confidence))
                overall_data[i].append(str(merged_xmin))
                overall_data[i].append(str(merged_ymin))
                overall_data[i].append(str(merged_xmax))
                overall_data[i].append(str(merged_ymax))
            CMD_result.append(overall_data[i])
        elif(data_number>2):
            while True:
                for k in range(len(data_list)):
                    #print(data_list)
                    merged = False
                    confidence_i, xmin_i, ymin_i, xmax_i, ymax_i = data_list[k]
                    for l in range(k):
                        confidence_j, xmin_j, ymin_j, xmax_j, ymax_j = data_list[l]
                        if (float(xmin_i) <= float(xmax_j) and float(xmax_i) >= float(xmin_j) and float(ymin_i) <= float(ymax_j) and float(ymax_i) >= float(ymin_j)):
                            merged_xmin = min(int(xmin_i), int(xmin_j))
                            merged_ymin = min(int(ymin_i), int(ymin_j))
                            merged_xmax = max(int(xmax_i), int(xmax_j))
                            merged_ymax = max(int(ymax_i), int(ymax_j))
                            merged_confidence = max(float(confidence_i), float(confidence_j))
                            data_list.remove(data_list[k])
                            data_list.remove(data_list[l])
                            data_list.append([str(merged_confidence),str(merged_xmin),str(merged_ymin),str(merged_xmax),str(merged_ymax)])
                            merged = True
                            break
                        else:
                            merged = False
                    if merged==True:
                        break
                if(len(data_list)==1):
                    break
                if merged==False:
                    break
            name = overall_data[i][0]
            overall_data[i].clear()
            overall_data[i].append(name)
            for k in range(len(data_list)):
                overall_data[i].append(str(data_list[k][0]))
                overall_data[i].append(str(data_list[k][1]))
                overall_data[i].append(str(data_list[k][2]))
                overall_data[i].append(str(data_list[k][3]))
                overall_data[i].append(str(data_list[k][4]))
            CMD_result.append(overall_data[i])
print(len(CMD_result)) 
print(CMD_result)           

106
[['20586908_6c613a14b80a8591_MG_R_CC_ANON', '0.56', '2093', '706', '2458', '958', '0.62', '2373', '1025', '2502', '1114', '0.68', '2441', '1612', '2593', '1763'], ['20586934_6c613a14b80a8591_MG_L_CC_ANON', '0.99', '92', '1986', '351', '2227'], ['20586960_6c613a14b80a8591_MG_R_ML_ANON', '0.9', '2441', '805', '2720', '985'], ['20586986_6c613a14b80a8591_MG_L_ML_ANON', '0.93', '61', '2295', '441', '2657'], ['20587612_f4b2d377f43ba0bd_MG_R_CC_ANON', '1.0', '2719', '502', '3076', '815'], ['20587664_f4b2d377f43ba0bd_MG_R_ML_ANON', '0.66', '916', '2869', '2020', '3694', '1.0', '2365', '845', '2760', '1152'], ['20587758_81cd83d2f4d78528_MG_L_CC_ANON', '0.51', '492', '-178', '1544', '660', '0.54', '191', '1084', '372', '1262'], ['20587810_81cd83d2f4d78528_MG_L_ML_ANON'], ['20587902_8dbbd4e51f549ff0_MG_R_CC_ANON', '0.68', '1411', '2376', '2550', '3163', '1.0', '1361', '1024', '2617', '1899'], ['20587928_8dbbd4e51f549ff0_MG_R_ML_ANON', '0.99', '1311', '1260', '2025', '2046'], ['20587994_024ee3

## CMD + FMD

### 將FMD重疊資料做處理

In [12]:
def calculate_overlap_percentage(box1, box2):

    x1_intersection = max(int(box1[1]), int(box2[1]))
    y1_intersection = max(int(box1[2]), int(box2[2]))
    x2_intersection = min(int(box1[3]), int(box2[3]))
    y2_intersection = min(int(box1[4]), int(box2[4]))

    area_box1 = (int(box1[3]) - int(box1[1])) * (int(box1[4]) - int(box1[2]))
    area_box2 = (int(box2[3]) - int(box2[1])) * (int(box2[4]) - int(box2[2]))

    intersection_area = max(0, int(x2_intersection) - int(x1_intersection)) * max(0, int(y2_intersection) - int(y1_intersection))

    union_area = area_box1 + area_box2 - intersection_area
    overlap_percentage = (intersection_area / union_area) * 100

    return overlap_percentage

overall_data = FMD_result
FMD_result = []
for i in range(len(overall_data)):
    data_number = int((len(overall_data[i])-1)/5)
    if(data_number<=1):
        FMD_result.append(overall_data[i])
        continue
    else:
        data_list = []
        num = 0
        for j in range(data_number):
            data_list.append([overall_data[i][1+num],overall_data[i][2+num],overall_data[i][3+num],overall_data[i][4+num],overall_data[i][5+num]])
            num = num + 5
        if data_number==2:
            confidence_i, xmin_i, ymin_i, xmax_i, ymax_i = data_list[0]
            confidence_j, xmin_j, ymin_j, xmax_j, ymax_j = data_list[1]
            if (float(xmin_i) <= float(xmax_j) and float(xmax_i) >= float(xmin_j) and float(ymin_i) <= float(ymax_j) and float(ymax_i) >= float(ymin_j)):
                if(float(calculate_overlap_percentage(data_list[0], data_list[1]))>40.0):
                    if(confidence_i>confidence_j):
                        name = overall_data[i][0]
                        overall_data[i].clear()
                        overall_data[i].append(name)
                        overall_data[i].append(confidence_i)
                        overall_data[i].append(xmin_i)
                        overall_data[i].append(ymin_i)
                        overall_data[i].append(xmax_i)
                        overall_data[i].append(ymax_i)
                        FMD_result.append(overall_data[i])
                    elif(confidence_i<confidence_j):
                        name = overall_data[i][0]
                        overall_data[i].clear()
                        overall_data[i].append(name)
                        overall_data[i].append(confidence_j)
                        overall_data[i].append(xmin_j)
                        overall_data[i].append(ymin_j)
                        overall_data[i].append(xmax_j)
                        overall_data[i].append(ymax_j)
                        FMD_result.append(overall_data[i])
                else:
                    FMD_result.append(overall_data[i])
            else:
                FMD_result.append(overall_data[i])
        elif(data_number>2):
            while True:
                for k in range(len(data_list)):
                    merged = False
                    confidence_i, xmin_i, ymin_i, xmax_i, ymax_i = data_list[k]
                    for l in range(k):
                        confidence_j, xmin_j, ymin_j, xmax_j, ymax_j = data_list[l]
                        if (float(xmin_i) <= float(xmax_j) and float(xmax_i) >= float(xmin_j) and float(ymin_i) <= float(ymax_j) and float(ymax_i) >= float(ymin_j)):
                            if(calculate_overlap_percentage(data_list[k], data_list[l])>40.0):
                                if(confidence_i>confidence_j):
                                    merged_xmin = xmin_i
                                    merged_ymin = ymin_i
                                    merged_xmax = xmax_i
                                    merged_ymax = ymax_i
                                    data_list.remove(data_list[k])
                                    data_list.remove(data_list[l])
                                    data_list.append([confidence_i,merged_xmin,merged_ymin,merged_xmax,merged_ymax])
                                    merged = True
                                    break
                                elif(confidence_j<confidence_i):
                                    merged_xmin = xmin_j
                                    merged_ymin = ymin_j
                                    merged_xmax = xmax_j
                                    merged_ymax = ymax_j
                                    data_list.remove(data_list[k])
                                    data_list.remove(data_list[l])
                                    data_list.append([confidence_j,merged_xmin,merged_ymin,merged_xmax,merged_ymax])
                                    merged = True
                                    break
                        else:
                            merged = False
                    if merged==True:
                        break
                if(len(data_list)==1):
                    merged = False
                if merged==False:
                    break
            name = overall_data[i][0]
            overall_data[i].clear()
            overall_data[i].append(name)
            for k in range(len(data_list)):
                overall_data[i].append(data_list[k][0])
                overall_data[i].append(data_list[k][1])
                overall_data[i].append(data_list[k][2])
                overall_data[i].append(data_list[k][3])
                overall_data[i].append(data_list[k][4])
            FMD_result.append(overall_data[i])
print(FMD_result)

[['20586908_6c613a14b80a8591_MG_R_CC_ANON'], ['20586934_6c613a14b80a8591_MG_L_CC_ANON', '0.84', '99', '2004', '330', '2234', '0.66', '616', '1130', '842', '1333'], ['20586960_6c613a14b80a8591_MG_R_ML_ANON', '0.26', '1795', '1275', '1898', '1408'], ['20586986_6c613a14b80a8591_MG_L_ML_ANON'], ['20587612_f4b2d377f43ba0bd_MG_R_CC_ANON', '0.68', '2769', '529', '3032', '767'], ['20587664_f4b2d377f43ba0bd_MG_R_ML_ANON', '0.55', '2378', '840', '2763', '1173'], ['20587758_81cd83d2f4d78528_MG_L_CC_ANON', '0.99', '188', '1068', '379', '1281'], ['20587810_81cd83d2f4d78528_MG_L_ML_ANON'], ['20587902_8dbbd4e51f549ff0_MG_R_CC_ANON', '0.98', '1528', '1084', '2411', '1752'], ['20587928_8dbbd4e51f549ff0_MG_R_ML_ANON', '0.94', '1366', '1279', '2108', '1987'], ['20587994_024ee3569b2605dc_MG_R_CC_ANON'], ['20588046_024ee3569b2605dc_MG_R_ML_ANON', '0.98', '1980', '2184', '2250', '2367'], ['20588190_8d0b9620c53c0268_MG_L_CC_ANON', '0.99', '83', '850', '289', '1105'], ['20588216_8d0b9620c53c0268_MG_L_ML_ANON'

### FMD和CMD資料合併

In [14]:
CMD_FMD_result = []
for i in range(len(FMD_result)):
    CMD_FMD_result.append([FMD_result[i][0]])

for k in range(len(CMD_FMD_result)):
    if(len(FMD_result[k])<6 and len(CMD_result[k])>=6):
        num = 0
        for i in range(int((len(CMD_result[k])-1)/5)):
            CMD_FMD_result[k].append(CMD_result[k][2+num])
            CMD_FMD_result[k].append(CMD_result[k][3+num])
            CMD_FMD_result[k].append(CMD_result[k][4+num])
            CMD_FMD_result[k].append(CMD_result[k][5+num])
            num = num + 5
    elif(len(CMD_result[k])<6 and len(FMD_result[k])>=6):
        num = 0
        for i in range(int((len(FMD_result[k])-1)/5)):
            CMD_FMD_result[k].append(FMD_result[k][2+num])
            CMD_FMD_result[k].append(FMD_result[k][3+num])
            CMD_FMD_result[k].append(FMD_result[k][4+num])
            CMD_FMD_result[k].append(FMD_result[k][5+num])
            num = num + 5      
    else:
        data_list = []
        num = 0
        for i in range(int((len(FMD_result[k])-1)/5)):
            data_list.append([FMD_result[k][1+num],FMD_result[k][2+num],FMD_result[k][3+num],FMD_result[k][4+num],FMD_result[k][5+num],"F"])
            num = num + 5
        num = 0
        for j in range(int((len(CMD_result[k])-1)/5)):
            data_list.append([CMD_result[k][1+num],CMD_result[k][2+num],CMD_result[k][3+num],CMD_result[k][4+num],CMD_result[k][5+num],"C"])
            num = num + 5
        if len(data_list)==2:
            confidence_i, xmin_i, ymin_i, xmax_i, ymax_i, index0 = data_list[0]
            confidence_j, xmin_j, ymin_j, xmax_j, ymax_j, index1 = data_list[1]
            if (float(xmin_i) <= float(xmax_j) and float(xmax_i) >= float(xmin_j) and float(ymin_i) <= float(ymax_j) and float(ymax_i) >= float(ymin_j)): 
                xmin_intersection = max(int(xmin_i), int(xmin_j))
                ymin_intersection = max(int(ymin_i), int(ymin_j))
                xmax_intersection = min(int(xmax_i), int(xmax_j))
                ymax_intersection = min(int(ymax_i), int(ymax_j))
                data_list.remove(data_list[0])
                data_list.remove(data_list[0])
                CMD_FMD_result[k].append(str(xmin_intersection))
                CMD_FMD_result[k].append(str(ymin_intersection))
                CMD_FMD_result[k].append(str(xmax_intersection))
                CMD_FMD_result[k].append(str(ymax_intersection))   
                  
        elif len(data_list)>2:
            while True:
                for i in range(len(data_list)):
                    merged = False
                    confidence_i, xmin_i, ymin_i, xmax_i, ymax_i, indexi = data_list[i]
                    for j in range(i):
                        confidence_j, xmin_j, ymin_j, xmax_j, ymax_j, indexj = data_list[j]
                        if (float(xmin_i) <= float(xmax_j) and float(xmax_i) >= float(xmin_j) and float(ymin_i) <= float(ymax_j) and float(ymax_i) >= float(ymin_j)):
                            confidence_intersection = max(confidence_i,confidence_j)
                            xmin_intersection = max(xmin_i, xmin_j)
                            ymin_intersection = max(ymin_i, ymin_j)
                            xmax_intersection = min(xmax_i, xmax_j)
                            ymax_intersection = min(ymax_i, ymax_j)
                            data_list.remove(data_list[i])
                            data_list.remove(data_list[j])
                            CMD_FMD_result[k].append(xmin_intersection)
                            CMD_FMD_result[k].append(ymin_intersection)
                            CMD_FMD_result[k].append(xmax_intersection)
                            CMD_FMD_result[k].append(ymax_intersection)
                            merged = True
                            break
                        else:
                            merged = False
                    if merged==True:
                        break
                if(len(data_list)==1 or len(data_list)==0):
                    break
                if merged==False:
                    break

print(CMD_FMD_result)    

[['20586908_6c613a14b80a8591_MG_R_CC_ANON', '2093', '706', '2458', '958', '2373', '1025', '2502', '1114', '2441', '1612', '2593', '1763'], ['20586934_6c613a14b80a8591_MG_L_CC_ANON', '99', '2004', '330', '2227'], ['20586960_6c613a14b80a8591_MG_R_ML_ANON'], ['20586986_6c613a14b80a8591_MG_L_ML_ANON', '61', '2295', '441', '2657'], ['20587612_f4b2d377f43ba0bd_MG_R_CC_ANON', '2769', '529', '3032', '767'], ['20587664_f4b2d377f43ba0bd_MG_R_ML_ANON', '2378', '845', '2760', '1152'], ['20587758_81cd83d2f4d78528_MG_L_CC_ANON', '191', '1084', '372', '1262'], ['20587810_81cd83d2f4d78528_MG_L_ML_ANON'], ['20587902_8dbbd4e51f549ff0_MG_R_CC_ANON', '1528', '1084', '2411', '1752'], ['20587928_8dbbd4e51f549ff0_MG_R_ML_ANON', '1366', '1279', '2025', '1987'], ['20587994_024ee3569b2605dc_MG_R_CC_ANON', '1929', '1921', '2205', '2174'], ['20588046_024ee3569b2605dc_MG_R_ML_ANON', '1980', '2184', '2250', '2367'], ['20588190_8d0b9620c53c0268_MG_L_CC_ANON', '83', '850', '289', '1099'], ['20588216_8d0b9620c53c0268_

### 計算FMD ACCURACY

In [15]:
def calculate_confusion_matrix(predictions, ground_truth_folder):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for prediction in predictions:
        predicted_boxes = []
        image_name = prediction[0]
        if len(prediction) < 5:
            ground_truth_file = os.path.join(ground_truth_folder, f"{image_name}.txt")
            
            if os.path.isfile(ground_truth_file):
                with open(ground_truth_file, 'r') as gt_file:
                    lines = gt_file.readlines()
                FN += len(lines) 
            else:
                TN += 1 
            
            continue
        coordinates = list(map(float, prediction[1:]))
        num_labels = len(coordinates) // 5    
        for i in range(num_labels):
            xmin, ymin, xmax, ymax = coordinates[i * 5 + 1], coordinates[i * 5 + 2], coordinates[i * 5 + 3], coordinates[i * 5 + 4]
            predicted_boxes.append((int(xmin),int(ymin),int(xmax),int(ymax)))

        ground_truth_file = os.path.join(ground_truth_folder, f"{image_name}.txt")
        image = os.path.join(image_folder, f"{image_name}.png")
        image = cv2.imread(image)
        height,width,_ = image.shape
        IMAGE_WIDTH = width  
        IMAGE_HEIGHT = height  
        
        if not os.path.isfile(ground_truth_file):
            print(f"找不到對應的ground truth檔案：{ground_truth_file}")
            continue

        with open(ground_truth_file, 'r') as gt_file:
            lines = gt_file.readlines()

        true_boxes = []

        for line in lines:
            parts = line.strip().split()
            x_center, y_center, width, height = map(float, parts[1:])

            xmin = int((x_center - width / 2) * IMAGE_WIDTH)
            ymin = int((y_center - height / 2) * IMAGE_HEIGHT)
            xmax = int((x_center + width / 2) * IMAGE_WIDTH)
            ymax = int((y_center + height / 2) * IMAGE_HEIGHT)

            true_boxes.append((xmin, ymin, xmax, ymax))
        
        for predicted_box in predicted_boxes:
            for true_box in true_boxes:
                iou = calculate_iou(predicted_box, true_box)

                if iou >= IOU_THRESHOLD:
                    TP += 1
                    break
            else:
                FP += 1

        for true_box in true_boxes:
            for predicted_box in predicted_boxes:
                iou = calculate_iou(predicted_box, true_box)

                if iou >= IOU_THRESHOLD:
                    break
            else:
                FN += 1

    return TP, TN, FP, FN

def calculate_iou(box1, box2):
    x1 = int(box1[0])
    y1 = int(box1[1])
    x2 = int(box1[2])
    y2 = int(box1[3])
    
    x3 = int(box2[0])
    y3 = int(box2[1])
    x4 = int(box2[2])
    y4 = int(box2[3])
    
    x_intersection = max(x1, x3)
    y_intersection = max(y1, y3)
    x_union = min(x2, x4)
    y_union = min(y2, y4)

    if x_intersection < x_union and y_intersection < y_union:
        intersection_area = (x_union - x_intersection) * (y_union - y_intersection)
        box1_area = (x2 - x1) * (y2 - y1)
        box2_area = (x4 - x3) * (y4 - y3)

        iou = intersection_area / (box1_area + box2_area - intersection_area)
        return iou

    return 0.0

IOU_THRESHOLD = 0.5 

predictions = FMD_result

ground_truth_folder = "C:/Users/HAUYU/Desktop/test_dataset/mass/with_label"
image_folder = "C:/Users/HAUYU/Desktop/data/full"

TP, TN, FP, FN = calculate_confusion_matrix(predictions, ground_truth_folder)
AP = TP/(TP+FP)
ACC = (TP+TN)/(TP+TN+FP+FN)

print(f"True Positives: {TP}")
print(f"True Negatives: {TN}")
print(f"False Positives: {FP}")
print(f"False Negatives: {FN}")
print(f"Accuracy: {ACC}")
print(f"Precision: {AP}")

True Positives: 89
True Negatives: 0
False Positives: 21
False Negatives: 26
Accuracy: 0.6544117647058824
Precision: 0.8090909090909091


### 計算FMD+CMD ACCURACY

In [16]:
def calculate_confusion_matrix(predictions, ground_truth_folder):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for prediction in predictions:
        predicted_boxes = []
        image_name = prediction[0]
        if len(prediction) < 5:
            ground_truth_file = os.path.join(ground_truth_folder, f"{image_name}.txt")
        
            if os.path.isfile(ground_truth_file):
                with open(ground_truth_file, 'r') as gt_file:
                    lines = gt_file.readlines()
                FN += len(lines)  
            else:
                TN += 1  
            
            continue
        coordinates = list(map(int, prediction[1:]))
        num_labels = len(coordinates) // 4    
        for i in range(num_labels):
            xmin, ymin, xmax, ymax = coordinates[i * 4], coordinates[i * 4 + 1], coordinates[i * 4 + 2], coordinates[i * 4 + 3]
            predicted_boxes.append((int(xmin),int(ymin),int(xmax),int(ymax)))

        ground_truth_file = os.path.join(ground_truth_folder, f"{image_name}.txt")
        image = os.path.join(image_folder, f"{image_name}.png")
        image = cv2.imread(image)
        height,width,_ = image.shape
        IMAGE_WIDTH = width 
        IMAGE_HEIGHT = height 
        
        if not os.path.isfile(ground_truth_file):
            print(f"找不到對應的ground truth檔案：{ground_truth_file}")
            continue

        with open(ground_truth_file, 'r') as gt_file:
            lines = gt_file.readlines()

        true_boxes = []

        for line in lines:
            parts = line.strip().split()
            x_center, y_center, width, height = map(float, parts[1:])

            xmin = int((x_center - width / 2) * IMAGE_WIDTH)
            ymin = int((y_center - height / 2) * IMAGE_HEIGHT)
            xmax = int((x_center + width / 2) * IMAGE_WIDTH)
            ymax = int((y_center + height / 2) * IMAGE_HEIGHT)

            true_boxes.append((xmin, ymin, xmax, ymax))

        for predicted_box in predicted_boxes:
            for true_box in true_boxes:
                iou = calculate_iou(predicted_box, true_box)

                if iou >= IOU_THRESHOLD:
                    TP += 1
                    break
            else:
                FP += 1

        for true_box in true_boxes:
            for predicted_box in predicted_boxes:
                iou = calculate_iou(predicted_box, true_box)

                if iou >= IOU_THRESHOLD:
                    break
            else:
                FN += 1

    return TP, TN, FP, FN

def calculate_iou(box1, box2):
    x1 = int(box1[0])
    y1 = int(box1[1])
    x2 = int(box1[2])
    y2 = int(box1[3])
    
    x3 = int(box2[0])
    y3 = int(box2[1])
    x4 = int(box2[2])
    y4 = int(box2[3])
    
    x_intersection = max(x1, x3)
    y_intersection = max(y1, y3)
    x_union = min(x2, x4)
    y_union = min(y2, y4)

    if x_intersection < x_union and y_intersection < y_union:
        intersection_area = (x_union - x_intersection) * (y_union - y_intersection)
        box1_area = (x2 - x1) * (y2 - y1)
        box2_area = (x4 - x3) * (y4 - y3)

        iou = intersection_area / (box1_area + box2_area - intersection_area)
        return iou

    return 0.0

IOU_THRESHOLD = 0.5 

predictions = CMD_FMD_result

ground_truth_folder = "C:/Users/HAUYU/Desktop/test_dataset/mass/with_label"
image_folder = "C:/Users/HAUYU/Desktop/data/full"

TP, TN, FP, FN = calculate_confusion_matrix(predictions, ground_truth_folder)
ACC = (TP+TN)/(TP+TN+FP+FN)
AP = TP/(TP+FP)

print(f"True Positives: {TP}")
print(f"True Negatives: {TN}")
print(f"False Positives: {FP}")
print(f"False Negatives: {FN}")
print(f"Accuracy: {ACC}")
print(f"Precision: {AP}")

True Positives: 87
True Negatives: 0
False Positives: 15
False Negatives: 28
Accuracy: 0.6692307692307692
Precision: 0.8529411764705882
